# 1. Camera check

In [5]:
import pyrealsense2 as rs
import numpy as np
import cv2
import time

# Создаём конвейер
pipe = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.color, 1920, 1080, rs.format.bgr8, 30)

# Запускаем
profile = pipe.start(cfg)

# Получаем настройки цветного сенсора
color_sensor = profile.get_device().query_sensors()[1]  # 0 — depth, 1 — color

# Включаем автоэкспозицию и автокоррекцию баланса белого
color_sensor.set_option(rs.option.enable_auto_exposure, True)
color_sensor.set_option(rs.option.enable_auto_white_balance, True)

# Можно слегка увеличить экспозицию вручную (по желанию)
# color_sensor.set_option(rs.option.exposure, 200)  # мс
# color_sensor.set_option(rs.option.gain, 32)      # усиление

# Настраиваем сглаживание
fps_counter, t0 = 0, time.time()

try:
    while True:
        frames = pipe.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue

        # Получаем numpy-кадр
        color_image = np.asanyarray(color_frame.get_data())

        # Лёгкое сглаживание для шумоподавления (уменьшает зерно)
        color_image = cv2.bilateralFilter(color_image, 5, 50, 50)

        # Считаем FPS
        fps_counter += 1
        if time.time() - t0 >= 1.0:
            fps = fps_counter
            fps_counter, t0 = 0, time.time()
        else:
            fps = None

        # Добавляем текст с FPS
        if fps is not None:
            cv2.putText(color_image, f"FPS: {fps}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Отображаем
        cv2.imshow("RGB 1920x1080 (smoothed)", color_image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    pipe.stop()
    cv2.destroyAllWindows()


# 2. Blue mask

In [8]:
import pyrealsense2 as rs
import numpy as np
import cv2

# --- Настройка камеры (только RGB 1920x1080) ---
pipe = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.color, 1920, 1080, rs.format.bgr8, 30)
pipe.start(cfg)

# --- Диапазон синего в HSV (смягчённый) ---
lower_blue = np.array([95, 60, 40])
upper_blue = np.array([145, 255, 255])

# Структурный элемент для морфологии
kernel = np.ones((7, 7), np.uint8)

try:
    while True:
        frames = pipe.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue

        # RGB-кадр
        color_image = np.asanyarray(color_frame.get_data())

        # 1️⃣ Гауссово размытие
        blurred = cv2.GaussianBlur(color_image, (9, 9), 0)

        # 2️⃣ HSV и маска
        hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv, lower_blue, upper_blue)

        # 3️⃣ Морфология: закрытие + открытие
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)

        # 4️⃣ Поиск связных областей
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)

        if num_labels > 1:
            # Индекс самой большой области (кроме фона)
            largest = 1 + np.argmax(stats[1:, cv2.CC_STAT_AREA])
            x, y, w, h, area = stats[largest]
            cx, cy = map(int, centroids[largest])

            # Создаём маску только для этой области
            mask = np.where(labels == largest, 255, 0).astype(np.uint8)

            # Визуализация
            cv2.rectangle(color_image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.circle(color_image, (cx, cy), 5, (0, 0, 255), -1)
            cv2.putText(color_image, f"Blue FinRay", (x, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

        # 5️⃣ Объединяем для визуализации
        mask_colored = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        result = cv2.bitwise_and(color_image, mask_colored)
        stacked = np.hstack((color_image, result))

        cv2.imshow("RGB | Masked", cv2.resize(stacked, (1280, 480)))

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    pipe.stop()
    cv2.destroyAllWindows()


# 3. Better mask

In [12]:
import pyrealsense2 as rs
import numpy as np
import cv2
import time

# ---------------- Config ----------------
CAM_W, CAM_H = 1280, 720      # capture resolution (HD)
PROC_W, PROC_H = 640, 360     # processing resolution (smaller = faster)

H_TOL = 20
S_TOL = 80
V_TOL = 80

MIN_AREA_PROC = 300

K_CLOSE = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
K_OPEN  = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
K_SMALL = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

# ---------------- Helpers ----------------
def make_hue_mask_wrap(hsv, low_h, high_h, low_s, high_s, low_v, high_v):
    lh = int(round(low_h)) % 180
    hh = int(round(high_h)) % 180
    ls = int(max(0, low_s)); hs = int(min(255, high_s))
    lv = int(max(0, low_v)); hv = int(min(255, high_v))
    if lh <= hh:
        return cv2.inRange(hsv, (lh, ls, lv), (hh, hs, hv))
    else:
        m1 = cv2.inRange(hsv, (lh, ls, lv), (179, hs, hv))
        m2 = cv2.inRange(hsv, (0, ls, lv), (hh, hs, hv))
        return cv2.bitwise_or(m1, m2)

def smooth_contour(contour, ksize=21, sigma=7):
    pts = contour.reshape(-1, 2).astype(np.float32)
    N = len(pts)
    if N < 6:
        return contour.astype(np.int32)
    pad = ksize // 2
    pts_padded = np.vstack([pts[-pad:], pts, pts[:pad]])
    gk = cv2.getGaussianKernel(ksize, sigma).flatten()
    gk /= gk.sum()
    xs = np.convolve(pts_padded[:,0], gk, mode='valid')
    ys = np.convolve(pts_padded[:,1], gk, mode='valid')
    sm = np.vstack([xs, ys]).T.astype(np.int32)
    return sm.reshape(-1,1,2)

# ---------------- Globals for mouse mapping ----------------
selected_hsv = None
color_selected = False
hsv_full = None          # updated each loop
# combined/display sizes used to map click coords -> full image coords
_combined_w = None
_combined_h = None
_display_w = None
_display_h = None

def on_mouse(event, x, y, flags, param):
    global selected_hsv, color_selected, hsv_full, _combined_w, _combined_h, _display_w, _display_h
    if event != cv2.EVENT_LBUTTONDOWN:
        return
    if hsv_full is None or _combined_w is None or _display_w is None:
        return
    # map click (x,y) from displayed (resized) image -> combined image coords
    # x_display in [0,_display_w) -> x_combined in [0,_combined_w)
    x_comb = int(x * (_combined_w / _display_w))
    y_comb = int(y * (_combined_h / _display_h))
    # determine which panel (0:left raw, 1:middle bbox, 2:right color_only)
    panel_idx = x_comb // CAM_W
    x_in_panel = x_comb % CAM_W
    y_in_panel = y_comb
    # clamp
    if x_in_panel < 0 or x_in_panel >= CAM_W or y_in_panel < 0 or y_in_panel >= CAM_H:
        return
    # read HSV from full-res hsv_full at that location
    h,s,v = hsv_full[y_in_panel, x_in_panel]
    selected_hsv = (int(h), int(s), int(v))
    color_selected = True
    print(f"[INFO] picked HSV={selected_hsv} at panel {panel_idx}, pixel ({x_in_panel},{y_in_panel})")

# ---------------- Camera init ----------------
pipe = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.color, CAM_W, CAM_H, rs.format.bgr8, 30)
profile = pipe.start(cfg)

WINDOW = "Color Picker - Click to pick color | Q to quit"
cv2.namedWindow(WINDOW, cv2.WINDOW_NORMAL)
cv2.resizeWindow(WINDOW, 1920, 720)

try:
    while True:
        frames = pipe.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue

        full = np.asanyarray(color_frame.get_data())  # CAM_H x CAM_W x 3
        # make hsv_full for picking color
        hsv_full = cv2.cvtColor(full, cv2.COLOR_BGR2HSV)

        # small copy for processing
        small = cv2.resize(full, (PROC_W, PROC_H), interpolation=cv2.INTER_LINEAR)
        blur = cv2.GaussianBlur(small, (5,5), 0)
        hsv_small = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

        raw_view = full.copy()
        bbox_view = full.copy()
        color_only = np.zeros_like(full)

        if color_selected and selected_hsv is not None:
            h0, s0, v0 = selected_hsv
            h_low, h_high = h0 - H_TOL, h0 + H_TOL
            s_low, s_high = s0 - S_TOL, s0 + S_TOL
            v_low, v_high = v0 - V_TOL, v0 + V_TOL

            mask_small = make_hue_mask_wrap(hsv_small, h_low, h_high, s_low, s_high, v_low, v_high)
            mask_small = cv2.morphologyEx(mask_small, cv2.MORPH_CLOSE, K_CLOSE, iterations=1)
            mask_small = cv2.morphologyEx(mask_small, cv2.MORPH_OPEN, K_OPEN, iterations=1)
            mask_small = cv2.medianBlur(mask_small, 5)

            # keep holes: find hierarchy and fill external contours, subtract holes
            contours, hierarchy = cv2.findContours(mask_small, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
            mask_final_small = np.zeros_like(mask_small)
            if contours and hierarchy is not None:
                hierarchy = hierarchy[0]
                # choose largest external component by filled area (external contours have parent == -1)
                external_idxs = [i for i in range(len(contours)) if hierarchy[i][3] == -1]
                if len(external_idxs) == 0:
                    # fallback: take the largest contour overall
                    idx_largest = int(np.argmax([cv2.contourArea(c) for c in contours]))
                    external_idxs = [idx_largest]
                # pick the largest external
                ext_areas = [(i, cv2.contourArea(contours[i])) for i in external_idxs]
                best_ext = max(ext_areas, key=lambda x: x[1])[0]
                # collect children (holes) recursively
                comp_indices = [best_ext]
                # BFS downwards to collect all descendants of best_ext
                q = [best_ext]
                while q:
                    p = q.pop(0)
                    for i in range(len(contours)):
                        if hierarchy[i][3] == p:
                            comp_indices.append(i)
                            q.append(i)
                # fill external contour (smoothed)
                outer_contour = contours[best_ext]
                outer_s = smooth_contour(outer_contour, ksize=21, sigma=7)
                cv2.fillPoly(mask_final_small, [outer_s], 255)
                # subtract holes (children in comp_indices except the outer itself)
                for idx in comp_indices:
                    if idx == best_ext:
                        continue
                    hole = contours[idx]
                    hole_s = smooth_contour(hole, ksize=21, sigma=7)
                    cv2.fillPoly(mask_final_small, [hole_s], 0)

            # remove tiny islands but preserve holes: small opening
            mask_final_small = cv2.morphologyEx(mask_final_small, cv2.MORPH_OPEN, K_SMALL, iterations=1)

            # scale mask up to full resolution using nearest to preserve holes
            mask_full = cv2.resize(mask_final_small, (CAM_W, CAM_H), interpolation=cv2.INTER_NEAREST)
            mask_full = (mask_full > 127).astype(np.uint8) * 255

            # find largest contour at full size for bbox/centroid
            cnts_full, _ = cv2.findContours(mask_full, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if cnts_full:
                c_full = max(cnts_full, key=cv2.contourArea)
                area_full = cv2.contourArea(c_full)
                if area_full > 0:
                    x,y,w,h = cv2.boundingRect(c_full)
                    M = cv2.moments(c_full)
                    if M["m00"] != 0:
                        cx = int(M["m10"]/M["m00"]); cy = int(M["m01"]/M["m00"])
                    else:
                        cx,cy = x + w//2, y + h//2
                    cv2.drawContours(bbox_view, [c_full], -1, (0,255,255), 2)
                    cv2.rectangle(bbox_view, (x,y), (x+w, y+h), (0,255,0), 3)
                    cv2.circle(bbox_view, (cx,cy), 6, (0,0,255), -1)
                    cv2.putText(bbox_view, f"HSV={selected_hsv}", (x, max(30, y-10)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)
                    color_only = cv2.bitwise_and(full, full, mask=mask_full)

        # combine three panels side-by-side (preserve original aspect ratio of each panel)
        combined = np.hstack([raw_view, bbox_view, color_only])  # width = 3*CAM_W, height = CAM_H
        _combined_h, _combined_w = combined.shape[0], combined.shape[1]

        # scale to fit large window height 720 while keeping aspect ratio
        target_h = 720
        scale = target_h / _combined_h
        display_w = int(_combined_w * scale)
        display_h = target_h
        display = cv2.resize(combined, (display_w, display_h), interpolation=cv2.INTER_AREA)

        # update mapping globals before showing (used by mouse handler)
        _display_w, _display_h = display_w, display_h

        hint = "Click (LMB) any pixel on camera to pick color | Q to quit"
        if color_selected and selected_hsv is not None:
            hint = f"{hint}    Selected HSV={selected_hsv}"
        cv2.putText(display, hint, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)

        cv2.imshow(WINDOW, display)
        cv2.setMouseCallback(WINDOW, on_mouse)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    pipe.stop()
    cv2.destroyAllWindows()


[INFO] picked HSV=(2, 234, 123) at panel 0, pixel (506,486)
[INFO] picked HSV=(72, 224, 108) at panel 0, pixel (594,254)
[INFO] picked HSV=(101, 255, 127) at panel 0, pixel (244,238)
[INFO] picked HSV=(0, 239, 157) at panel 0, pixel (506,344)
[INFO] picked HSV=(100, 232, 147) at panel 0, pixel (334,572)
[INFO] picked HSV=(1, 238, 162) at panel 0, pixel (496,342)
[INFO] picked HSV=(72, 173, 121) at panel 0, pixel (690,182)
[INFO] picked HSV=(100, 199, 122) at panel 0, pixel (350,218)
[INFO] picked HSV=(173, 8, 129) at panel 0, pixel (224,202)
[INFO] picked HSV=(102, 255, 118) at panel 0, pixel (234,480)
[INFO] picked HSV=(3, 201, 185) at panel 0, pixel (452,330)
[INFO] picked HSV=(70, 223, 103) at panel 0, pixel (588,262)
[INFO] picked HSV=(166, 174, 98) at panel 0, pixel (506,318)
[INFO] picked HSV=(100, 139, 66) at panel 0, pixel (922,620)
[INFO] picked HSV=(73, 243, 104) at panel 0, pixel (730,384)
[INFO] picked HSV=(71, 222, 107) at panel 0, pixel (544,224)
[INFO] picked HSV=(15, 25

# 4. Point cloud fill

In [4]:
import pyrealsense2 as rs
import numpy as np
import cv2
import time

# ---------------- Config ----------------
CAM_W, CAM_H = 1280, 720      # capture resolution (HD)
PROC_W, PROC_H = 640, 360     # processing resolution (smaller = faster)

# HSV tolerance (around picked HSV)
H_TOL = 18
S_TOL = 80
V_TOL = 80

MIN_AREA_PROC = 300
SAMPLE_COUNT = 20             # default sample count (trackbar controls)

# Morphology
K_CLOSE = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9, 9))
K_OPEN  = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
K_SMALL = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))

# ---------------- Drift/stability params ----------------
# If median motion is below this (in pixels), treat as jitter and don't snap fully.
JITTER_THRESH_PX = 1.0   # pixels
# Exponential smoothing for small jitter vs real motion
EMA_ALPHA_MOVING = 0.99   # when real motion — respond fairly quickly
EMA_ALPHA_STATIC = 0.1  # when static / jitter — update slowly
# When applying registration fallback (affine), blend with this weight
REG_ALPHA = 0.6

# optical flow params
LK_WIN = (21, 21)
LK_MAXLEVEL = 3
LK_CRITERIA = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01)

# RNG
rng = np.random.default_rng()

# ---------------- Helpers ----------------
def make_hue_mask_wrap(hsv, low_h, high_h, low_s, high_s, low_v, high_v):
    lh = int(round(low_h)) % 180
    hh = int(round(high_h)) % 180
    ls = int(max(0, low_s)); hs = int(min(255, high_s))
    lv = int(max(0, low_v)); hv = int(min(255, high_v))
    if lh <= hh:
        return cv2.inRange(hsv, (lh, ls, lv), (hh, hs, hv))
    else:
        m1 = cv2.inRange(hsv, (lh, ls, lv), (179, hs, hv))
        m2 = cv2.inRange(hsv, (0, ls, lv), (hh, hs, hv))
        return cv2.bitwise_or(m1, m2)

def smooth_contour(contour, ksize=21, sigma=7):
    pts = contour.reshape(-1, 2).astype(np.float32)
    N = len(pts)
    if N < 6:
        return contour.astype(np.int32)
    pad = ksize // 2
    pts_padded = np.vstack([pts[-pad:], pts, pts[:pad]])
    gk = cv2.getGaussianKernel(ksize, sigma).flatten()
    gk /= gk.sum()
    xs = np.convolve(pts_padded[:,0], gk, mode='valid')
    ys = np.convolve(pts_padded[:,1], gk, mode='valid')
    sm = np.vstack([xs, ys]).T.astype(np.int32)
    return sm.reshape(-1,1,2)

# ---------------- Globals for UI / tracking ----------------
selected_hsv = None
color_selected = False
hsv_full = None

sampled_points = None            # Nx2 float32 (full-res coordinates)
sampled_points_initial = None
prev_gray = None
_tracking_initialized = False

# gui mapping globals
_combined_w = None
_combined_h = None
_display_w = None
_display_h = None

# ---------------- Mouse callback ----------------
def on_mouse(event, x, y, flags, param):
    global selected_hsv, color_selected, hsv_full
    global sampled_points, sampled_points_initial, _tracking_initialized
    global _combined_w, _combined_h, _display_w, _display_h
    if event != cv2.EVENT_LBUTTONDOWN:
        return
    if hsv_full is None or _combined_w is None or _display_w is None or _display_w == 0:
        return
    # Map click from displayed coords -> combined coords
    x_comb = int(x * (_combined_w / _display_w))
    y_comb = int(y * (_combined_h / _display_h))
    # Panel index (left raw panel is panel 0)
    panel_idx = x_comb // CAM_W
    x_in_panel = x_comb % CAM_W
    y_in_panel = y_comb
    # clamp
    if not (0 <= x_in_panel < CAM_W and 0 <= y_in_panel < CAM_H):
        return
    h,s,v = hsv_full[y_in_panel, x_in_panel]
    selected_hsv = (int(h), int(s), int(v))
    color_selected = True
    # force regeneration of points on click
    _tracking_initialized = False
    sampled_points = None
    sampled_points_initial = None
    print(f"[INFO] Selected HSV={selected_hsv} at ({x_in_panel},{y_in_panel}) panel={panel_idx}")

# ---------------- Trackbar callback ----------------
def on_trackbar(val):
    # no immediate re-generation; regeneration happens on next click
    pass

# ---------------- Camera init & GUI ----------------
pipe = rs.pipeline()
cfg = rs.config()
cfg.enable_stream(rs.stream.color, CAM_W, CAM_H, rs.format.bgr8, 30)
profile = pipe.start(cfg)

WINDOW = "Color Tracker + Stable Points (click to pick color) | Q to quit"
cv2.namedWindow(WINDOW, cv2.WINDOW_NORMAL)
cv2.resizeWindow(WINDOW, 1920, 720)
cv2.setMouseCallback(WINDOW, on_mouse)

# create trackbar for sample count
cv2.createTrackbar("NumPoints", WINDOW, SAMPLE_COUNT, 5000, on_trackbar)

# ---------------- Main loop ----------------
try:
    while True:
        frames = pipe.wait_for_frames()
        color_frame = frames.get_color_frame()
        if not color_frame:
            continue

        full = np.asanyarray(color_frame.get_data())  # CAM_H x CAM_W x 3
        hsv_full = cv2.cvtColor(full, cv2.COLOR_BGR2HSV)

        # small copy for processing
        small = cv2.resize(full, (PROC_W, PROC_H), interpolation=cv2.INTER_LINEAR)
        blur = cv2.GaussianBlur(small, (5,5), 0)
        hsv_small = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

        raw_view = full.copy()
        bbox_view = full.copy()
        color_only = np.zeros_like(full)

        # read SAMPLE_COUNT from trackbar (it won't regenerate points until next click)
        SAMPLE_COUNT = cv2.getTrackbarPos("NumPoints", WINDOW)

        if color_selected and selected_hsv is not None:
            h0, s0, v0 = selected_hsv
            h_low, h_high = h0 - H_TOL, h0 + H_TOL
            s_low, s_high = s0 - S_TOL, s0 + S_TOL
            v_low, v_high = v0 - V_TOL, v0 + V_TOL

            # small mask
            mask_small = make_hue_mask_wrap(hsv_small, h_low, h_high, s_low, s_high, v_low, v_high)
            mask_small = cv2.morphologyEx(mask_small, cv2.MORPH_CLOSE, K_CLOSE, iterations=1)
            mask_small = cv2.morphologyEx(mask_small, cv2.MORPH_OPEN, K_OPEN, iterations=1)
            mask_small = cv2.medianBlur(mask_small, 5)

            # preserve holes: pick largest external component and subtract holes
            contours, hierarchy = cv2.findContours(mask_small, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
            mask_final_small = np.zeros_like(mask_small)
            if contours and hierarchy is not None:
                hierarchy = hierarchy[0]
                external_idxs = [i for i in range(len(contours)) if hierarchy[i][3] == -1]
                if len(external_idxs) == 0:
                    idx_largest = int(np.argmax([cv2.contourArea(c) for c in contours]))
                    external_idxs = [idx_largest]
                ext_areas = [(i, cv2.contourArea(contours[i])) for i in external_idxs]
                best_ext = max(ext_areas, key=lambda x: x[1])[0]

                # collect descendants
                comp_indices = [best_ext]
                q = [best_ext]
                while q:
                    p = q.pop(0)
                    for i in range(len(contours)):
                        if hierarchy[i][3] == p:
                            comp_indices.append(i)
                            q.append(i)

                outer_contour = contours[best_ext]
                outer_s = smooth_contour(outer_contour, ksize=21, sigma=7)
                cv2.fillPoly(mask_final_small, [outer_s], 255)

                for idx in comp_indices:
                    if idx == best_ext:
                        continue
                    hole = contours[idx]
                    hole_s = smooth_contour(hole, ksize=21, sigma=7)
                    cv2.fillPoly(mask_final_small, [hole_s], 0)

            mask_final_small = cv2.morphologyEx(mask_final_small, cv2.MORPH_OPEN, K_SMALL, iterations=1)

            # upscale mask
            mask_full = cv2.resize(mask_final_small, (CAM_W, CAM_H), interpolation=cv2.INTER_NEAREST)
            mask_full = (mask_full > 127).astype(np.uint8) * 255

            # initialize sampled_points on click (only once until next click)
            if (not _tracking_initialized) and np.any(mask_full > 0):
                ys, xs = np.where(mask_full > 0)
                if len(xs) > 0 and SAMPLE_COUNT > 0:
                    count = min(SAMPLE_COUNT, len(xs))
                    replace = False if count <= len(xs) else True
                    idxs = rng.choice(len(xs), size=count, replace=replace)
                    pts = np.column_stack([xs[idxs], ys[idxs]]).astype(np.float32)
                    sampled_points = pts.copy()
                    sampled_points_initial = pts.copy()
                else:
                    sampled_points = np.zeros((0,2), dtype=np.float32)
                    sampled_points_initial = sampled_points.copy()
                prev_gray = cv2.cvtColor(full, cv2.COLOR_BGR2GRAY)
                _tracking_initialized = True
                print(f"[INFO] Initialized {len(sampled_points)} points.")

            # If points exist, track them with LK + stability filtering
            if _tracking_initialized and sampled_points is not None and sampled_points.shape[0] > 0:
                curr_gray = cv2.cvtColor(full, cv2.COLOR_BGR2GRAY)
                p0 = sampled_points.reshape(-1,1,2).astype(np.float32)
                p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, p0, None,
                                                       winSize=LK_WIN, maxLevel=LK_MAXLEVEL,
                                                       criteria=LK_CRITERIA)
                if p1 is not None:
                    st = st.reshape(-1)
                    good_idx = np.where(st==1)[0]
                    if good_idx.size > 0:
                        # motion vectors only for successfully tracked points
                        deltas = (p1[good_idx,0] - p0[good_idx,0])  # Nx2
                        d_norms = np.linalg.norm(deltas, axis=1)
                        median_motion = float(np.median(d_norms))
                    else:
                        median_motion = np.inf

                    # If median motion small -> consider it jitter; use small alpha smoothing
                    if median_motion < JITTER_THRESH_PX:
                        alpha = EMA_ALPHA_STATIC
                    else:
                        alpha = EMA_ALPHA_MOVING

                    # Update positions: for tracked points, apply EMA blending; for lost points keep previous
                    if p1 is not None:
                        for i_idx in range(len(p0)):
                            if st[i_idx] == 1:
                                new_pt = p1[i_idx,0]
                                sampled_points[i_idx] = (1.0 - alpha) * sampled_points[i_idx] + alpha * new_pt
                            else:
                                # leave as is (or optionally snap to nearest later)
                                pass

                # If too many points outside the current mask, attempt a robust registration (affine) to pull cloud back
                inside_mask_flags = []
                ys_mask, xs_mask = np.where(mask_full > 0)
                mask_points = None
                if len(xs_mask) > 0:
                    mask_points = np.column_stack([xs_mask, ys_mask]).astype(np.float32)
                for pt in sampled_points:
                    x_pt = int(round(pt[0])); y_pt = int(round(pt[1]))
                    inside_mask_flags.append(0 <= x_pt < CAM_W and 0 <= y_pt < CAM_H and mask_full[y_pt, x_pt] > 0)
                inside_count = sum(1 for v in inside_mask_flags if v)
                if inside_count < max(3, sampled_points.shape[0]//2) and mask_points is not None and mask_points.shape[0] > 0:
                    # Build simple nearest correspondence between current sampled_points and mask_points (subsample masks for speed)
                    max_target = 3000
                    if mask_points.shape[0] > max_target:
                        idxs_t = rng.choice(len(mask_points), size=max_target, replace=False)
                        pts_target_sub = mask_points[idxs_t]
                    else:
                        pts_target_sub = mask_points
                    # brute-force nearest neighbor matching
                    src = sampled_points.copy()
                    matched_src = []
                    matched_dst = []
                    for p in src:
                        d2 = np.sum((pts_target_sub - p.reshape(1,2))**2, axis=1)
                        idx_min = int(np.argmin(d2))
                        matched_src.append(p)
                        matched_dst.append(pts_target_sub[idx_min])
                    matched_src = np.array(matched_src, dtype=np.float32)
                    matched_dst = np.array(matched_dst, dtype=np.float32)
                    if matched_src.shape[0] >= 3:
                        M, inliers = cv2.estimateAffinePartial2D(matched_src, matched_dst, method=cv2.RANSAC,
                                                                 ransacReprojThreshold=5.0, maxIters=2000)
                        if M is not None:
                            pts_h = np.hstack([sampled_points, np.ones((sampled_points.shape[0],1), dtype=np.float32)])
                            transformed = (M @ pts_h.T).T
                            # blend to avoid sudden jumps (REG_ALPHA)
                            sampled_points = (1.0 - REG_ALPHA) * sampled_points + REG_ALPHA * transformed.astype(np.float32)

                # Ensure points remain inside mask; if not, snap to nearest mask pixel (this is expensive but used rarely)
                if mask_points is not None and mask_points.shape[0] > 0:
                    for i_pt, pt in enumerate(sampled_points):
                        x_pt = int(round(pt[0])); y_pt = int(round(pt[1]))
                        if not (0 <= x_pt < CAM_W and 0 <= y_pt < CAM_H and mask_full[y_pt, x_pt] > 0):
                            # nearest mask pixel brute force
                            d2 = (mask_points[:,0] - pt[0])**2 + (mask_points[:,1] - pt[1])**2
                            idxn = int(np.argmin(d2))
                            sampled_points[i_pt] = mask_points[idxn].astype(np.float32)

                prev_gray = curr_gray.copy()

            # Visualization: draw sampled_points on bbox_view and color_only
            if sampled_points is not None and sampled_points.shape[0] > 0:
                for (px,py) in sampled_points.astype(np.int32):
                    cv2.circle(bbox_view, (int(px), int(py)), 4, (0,255,0), -1)
                color_only = cv2.bitwise_and(full, full, mask=mask_full)
                for (px,py) in sampled_points.astype(np.int32):
                    cv2.circle(color_only, (int(px), int(py)), 4, (0,255,0), -1)
            else:
                color_only = cv2.bitwise_and(full, full, mask=mask_full)

            # draw bbox/contour/centroid
            cnts_full, _ = cv2.findContours(mask_full, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            if cnts_full:
                c_full = max(cnts_full, key=cv2.contourArea)
                area_full = cv2.contourArea(c_full)
                if area_full > 0:
                    x,y,w,h = cv2.boundingRect(c_full)
                    M = cv2.moments(c_full)
                    if M["m00"] != 0:
                        cx = int(M["m10"]/M["m00"]); cy = int(M["m01"]/M["m00"])
                    else:
                        cx,cy = x + w//2, y + h//2
                    cv2.drawContours(bbox_view, [c_full], -1, (0,255,255), 2)
                    cv2.rectangle(bbox_view, (x,y), (x+w, y+h), (0,255,0), 3)
                    cv2.circle(bbox_view, (cx,cy), 6, (0,0,255), -1)
                    cv2.putText(bbox_view, f"HSV={selected_hsv}", (x, max(30, y-10)),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2, cv2.LINE_AA)

        # combine three panels
        combined = np.hstack([raw_view, bbox_view, color_only])
        _combined_h, _combined_w = combined.shape[0], combined.shape[1]

        # scale to fit 720px height
        target_h = 720
        scale = target_h / _combined_h
        display_w = int(_combined_w * scale)
        display_h = target_h
        display = cv2.resize(combined, (display_w, display_h), interpolation=cv2.INTER_AREA)

        # update mapping globals used for clicks
        _display_w, _display_h = display_w, display_h

        # update SAMPLE_COUNT from trackbar
        SAMPLE_COUNT = cv2.getTrackbarPos("NumPoints", WINDOW)

        hint = f"Click to pick color (regenerates points) | Q to quit | NumPoints={SAMPLE_COUNT}"
        if color_selected and selected_hsv is not None:
            hint = f"{hint}    Selected HSV={selected_hsv}"
        cv2.putText(display, hint, (10,30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2, cv2.LINE_AA)

        cv2.imshow(WINDOW, display)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

finally:
    pipe.stop()
    cv2.destroyAllWindows()


[INFO] Selected HSV=(71, 240, 102) at (392,378) panel=1
[INFO] Initialized 20 points.
[INFO] Selected HSV=(72, 201, 123) at (350,332) panel=1
[INFO] Initialized 574 points.
[INFO] Selected HSV=(71, 213, 121) at (277,336) panel=1
[INFO] Initialized 1485 points.
[INFO] Selected HSV=(73, 220, 115) at (404,353) panel=1
[INFO] Initialized 5000 points.
